In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from super_cub import *

In [ ]:
alphas = np.linspace(-6.27, 15, 100)
del_e = np.linspace(-10, 10, 5)
Re_c = 500000
airplane = super_cub('xf-n2415-il-500000.csv')

Cms = np.zeros((len(del_e), len(alphas)))
Cls = np.zeros((len(del_e), len(alphas)))
Cds = np.zeros((len(del_e), len(alphas)))

for i in range(len(del_e)):
    for j in range(len(alphas)):
        Cms[i,j] = airplane.get_coefficients(alphas[j], del_e[i], Re_c, 0.4)[0]
        Cls[i,j] = airplane.get_coefficients(alphas[j], del_e[i], Re_c, 0.4)[1]
        Cds[i,j] = airplane.get_coefficients(alphas[j], del_e[i], Re_c, 0.4)[2]
    
plt.style.use('default')   # Set the aesthetic style of the plots

# Define the del_e values
del_e_values = np.linspace(-10, 10, 5)

fig = plt.figure()
# Plot Cm vs alphas for each del_e value
for i, del_e in enumerate(del_e_values):
    plt.plot(alphas, Cms[i], label=f"$\delta_e$ = {del_e}")
    
# Set the plot title and labels
plt.title("$C_m$ vs $\\alpha$")
plt.xlabel("$\\alpha$")
plt.ylabel("$C_m$")

# Add a legend
plt.legend()
plt.grid(True)
# Show the plot
plt.savefig('Cm_vs_alpha.jpg', dpi=300)
plt.show()

fig = plt.figure()
# Plot Cl vs alphas for each del_e value
for i, del_e in enumerate(del_e_values):
    plt.plot(alphas, Cls[i], label=f"$\delta_e$ = {del_e}")
    
# Set the plot title and labels
plt.title("$C_l$ vs $\\alpha$")
plt.xlabel("$\\alpha$")
plt.ylabel("$C_l$")

# Add a legend
plt.legend()
plt.grid(True)
# Show the plot
plt.savefig('Cl_vs_alpha.jpg', dpi=300)
plt.show()

fig = plt.figure()
# Plot Cl vs Cd for each del_e value
for i, del_e in enumerate(del_e_values):
    plt.plot(Cds[i], Cls[i], label=f"$\delta_e$ = {del_e}")
    
# Set the plot title and labels
plt.title("$C_l$ vs $C_d$")
plt.xlabel("$C_d$")
plt.ylabel("$C_l$")

# Add a legend
plt.legend()
plt.grid(True)
# Show the plot
plt.savefig('Cl_vs_Cd.jpg', dpi=300)
plt.show()

In [ ]:
from scipy.optimize import fsolve

alpha_trim = np.linspace(-18,18, 100)
trim_cl = []
trim_cd = []

for alpha in alpha_trim:
    f_CM = lambda x: airplane.get_coefficients(5, x, Re_c, 0.4)[0]
    del_e = fsolve(f_CM, 0)[0]
    trim_cl.append(airplane.get_coefficients(alpha, del_e, Re_c, 0.4)[1])
    trim_cd.append(airplane.get_coefficients(alpha, del_e, Re_c, 0.4)[2])


In [ ]:
from scipy.optimize import curve_fit

def parabolic_function(x, cd0, K):
    return cd0 + K*x**2

popt, _ = curve_fit(parabolic_function, trim_cl, trim_cd, bounds = ([0.025, -np.inf],[np.inf, 0.15]))
print(popt)

In [ ]:
%matplotlib widget
# Fit the data to a quadratic equation
coefficients = np.polyfit(trim_cl, trim_cd, 2)

# Extract the coefficients
cd0 = popt[0]
K = popt[1]

# Generate the fitted curve
fit_curve = np.polyval(coefficients, trim_cl)

fig = plt.figure()
# Plot the data and the fitted curve
plt.plot(trim_cd, trim_cl, label='Data')
plt.plot(cd0+K*np.array(trim_cl)**2, trim_cl, label='Fitted Curve')

# Annotate the fit equations
equation = f"$C_d = {cd0:.5f} + {K:.5f}C_l^2$"
plt.annotate(equation, xy=(0.5, 0.5), xycoords='axes fraction', fontsize=12)

# Set the plot title and labels
plt.title("Trimmed Drag Polar with Fitted Curve")
plt.xlabel("$C_d$")
plt.ylabel("$C_l$")

# Add a legend
plt.legend()

# Show the plot
plt.grid(True)
plt.savefig('Trimmed_Drag_Polar.jpg', dpi=300)
plt.show()


In [ ]:
v = np.linspace(6, 14, 100)
rho = 1.22
thrust = cd0*rho*v**2*airplane.Sw/2 + 2*K*(0.75*9.81)**2/(rho*v**2*airplane.Sw)
power = cd0*rho*v**3*airplane.Sw/2 + 2*K*(0.75*9.81)**2/(rho*v*airplane.Sw)

gram_thrust = thrust/(9.81)*1000



In [ ]:
# Find the index of the minimum gram thrust value
fig, ax1 = plt.subplots()
min_index = np.argmin(gram_thrust)

# Annotate the minimum gram thrust value
ax1.annotate(f"Velocity for min thrust: {v[min_index]:.2f} m/s", 
             xy=(v[min_index], gram_thrust[min_index]), 
             xytext=(v[min_index]-2, gram_thrust[min_index]+25), 
             arrowprops=dict(facecolor='black', arrowstyle='->'))


# Plot gram thrust
ax1.plot(v, gram_thrust, 'b-', label='Gram Thrust')
ax1.set_xlabel('Velocity (m/s)')
ax1.set_ylabel('Thrust (grams)', color='b')
ax1.tick_params('y', colors='b')

# Create a second y-axis
ax2 = ax1.twinx()

# Plot power
ax2.plot(v, power, 'r-', label='Power')
ax2.set_ylabel('Power', color='r')
ax2.tick_params('y', colors='r')

# Add a legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper right')

# Set the plot title
plt.title('Thrust and Power vs Velocity')

# Show the plot
plt.grid(True)
plt.savefig('Thrust_and_Power_vs_Velocity.jpg', dpi=300)
plt.show()


In [ ]:
thrust1 = cd0*rho*v**2*airplane.Sw/2
thrust2 = 2*K*(0.75*9.81)**2/(rho*v**2*airplane.Sw)

import matplotlib.pyplot as plt

fig = plt.figure()
# Plot thrust1 and thrust2 against v
plt.plot(v, thrust1, label='Thrust 1')
plt.plot(v, thrust2, label='Thrust 2')
plt.plot(v, thrust, label='Thrust')

# Set the plot title and labels
plt.title('Thrust 1 and Thrust 2 vs Velocity')
plt.xlabel('Velocity (m/s)')
plt.ylabel('Thrust')
plt.legend()

# Show the plot
plt.grid(True)
plt.show()
